#Terceira Tentativa do Processo de ETL com Dados da RAIS


#Processo de ETL com Dados da RAIS

In [ ]:
import pandas as pd

caminho_do_arquivo = '/content/RAIS_VINC_PUB_NORTE.txt'

try:
    dados = pd.read_csv(caminho_do_arquivo, sep=';', encoding='latin1', low_memory=False)
    print('Arquivo lido com sucesso!')
except Exception as e:
    print(f'Erro ao ler o arquivo: {e}')

Arquivo lido com sucesso!


In [ ]:
print(dados.columns)

Index(['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'Causa Afastamento 1',
       'Causa Afastamento 2', 'Causa Afastamento 3', 'Motivo Desligamento',
       'CBO Ocupação 2002', 'CNAE 2.0 Classe', 'CNAE 95 Classe',
       'Distritos SP', 'Vínculo Ativo 31/12', 'Faixa Etária',
       'Faixa Hora Contrat', 'Faixa Remun Dezem (SM)',
       'Faixa Remun Média (SM)', 'Faixa Tempo Emprego',
       'Escolaridade após 2005', 'Qtd Hora Contr', 'Idade',
       'Ind CEI Vinculado', 'Ind Simples', 'Mês Admissão', 'Mês Desligamento',
       'Mun Trab', 'Município', 'Nacionalidade', 'Natureza Jurídica',
       'Ind Portador Defic', 'Qtd Dias Afastamento', 'Raça Cor',
       'Regiões Adm DF', 'Vl Remun Dezembro Nom', 'Vl Remun Dezembro (SM)',
       'Vl Remun Média Nom', 'Vl Remun Média (SM)', 'CNAE 2.0 Subclasse',
       'Sexo Trabalhador', 'Tamanho Estabelecimento', 'Tempo Emprego',
       'Tipo Admissão', 'Tipo Estab', 'Tipo Estab.1', 'Tipo Defic',
       'Tipo Vínculo', 'IBGE Subsetor', 'Vl 

In [ ]:
dados_filtrados = dados[dados['Vínculo Ativo 31/12'] != 0]
print(f'Registros antes da filtragem: {len(dados)}, após filtragem: {len(dados_filtrados)}')

Registros antes da filtragem: 300819, após filtragem: 157636


In [ ]:
dados_filtrados['DIV CNAE 20'] = dados_filtrados['CNAE 2.0 Classe'].astype(str).apply(lambda x: x[:2] if pd.notnull(x) else x)
print('Nova variável criada com sucesso!')

Nova variável criada com sucesso!


<ipython-input-5-0e09a115135c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_filtrados['DIV CNAE 20'] = dados_filtrados['CNAE 2.0 Classe'].astype(str).apply(lambda x: x[:2] if pd.notnull(x) else x)


In [ ]:
print(dados_filtrados.dtypes)

Bairros SP                object
Bairros Fortaleza         object
Bairros RJ                object
Causa Afastamento 1        int64
Causa Afastamento 2        int64
                          ...   
Vl Rem Novembro SC        object
Ano Chegada Brasil       float64
Ind Trab Intermitente    float64
Ind Trab Parcial         float64
DIV CNAE 20               object
Length: 61, dtype: object


In [ ]:
max_length_bairros_sp = dados_filtrados['Bairros SP'].astype(str).map(len).max()
max_length_bairros_fortaleza = dados_filtrados['Bairros Fortaleza'].astype(str).map(len).max()
max_length_bairros_rj = dados_filtrados['Bairros RJ'].astype(str).map(len).max()
max_length_div_cnae_20 = dados_filtrados['DIV CNAE 20'].astype(str).map(len).max()

print(f"Comprimento máximo para Bairros SP: {max_length_bairros_sp}")
print(f"Comprimento máximo para Bairros Fortaleza: {max_length_bairros_fortaleza}")
print(f"Comprimento máximo para Bairros RJ: {max_length_bairros_rj}")
print(f"Comprimento máximo para DIV CNAE 20: {max_length_div_cnae_20}")

Comprimento máximo para Bairros SP: 20
Comprimento máximo para Bairros Fortaleza: 20
Comprimento máximo para Bairros RJ: 20
Comprimento máximo para DIV CNAE 20: 2


#Analisando Layout

In [ ]:
import pandas as pd

layout_path = '/content/RAIS_vinculos_layout2020.xls'

# Tentar ler o arquivo de layout
try:
    layout_df = pd.read_excel(layout_path, sheet_name=0)
    print(layout_df.head())
except Exception as e:
    print(f"Erro ao ler o arquivo: {e}")

                                  RAIS TRABALHADOR    \
0     (arquivos com separador ponto e virgula - ";")   
1  OBS: ao encontrar dados como "-1",  com ou sem...   
2                                               Nome   
3                                          BAIRRO_SP   
4                                        BAIRRO FORT   

                          Unnamed: 1         Unnamed: 2  \
0                                NaN                NaN   
1                                NaN                NaN   
2              Descricao da Variável  Tamanho(em bytes)   
3  Bairros do Municipio de São Paulo                  4   
4  Bairros do município de Fortaleza                  4   

                                          Unnamed: 3      Unnamed: 4  \
0                                                NaN             NaN   
1                                                NaN             NaN   
2                                         Categorias  Valor na Fonte   
3  (ver planilha cor

Com base no resultado fornecido, agora podemos entender a estrutura do layout do arquivo da RAIS para trabalhadores.

- **Nome**: A primeira coluna indica o nome da variável tal como aparecerá no arquivo de dados. Por exemplo, `BAIRRO_SP` e `BAIRRO FORT` se referem a bairros do município de São Paulo e Fortaleza, respectivamente.
- **Descrição da Variável**: A segunda coluna fornece uma descrição mais detalhada de cada variável.
- **Tamanho (em bytes)**: A terceira coluna indica o tamanho de cada variável em bytes.
- **Categorias**: A quarta coluna menciona que para algumas variáveis, como bairros, pode haver uma planilha correspondente contendo mais detalhes, como categorias ou códigos utilizados para representar os dados.


# Conectando ao Banco

In [ ]:
!pip install sqlalchemy pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00


In [ ]:
from sqlalchemy import create_engine

In [ ]:
# Certifique-se de usar os nomes de variáveis corretos
connection_string = f"mysql+pymysql://{usuario}:{senha}@{host}:{porta}/{database}"

# Criar o engine de conexão
engine = create_engine(connection_string)

# Testar a conexão
try:
    with engine.connect() as connection:
        result = connection.execute("SELECT 1")
        for row in result:
            print(row)  # Deve imprimir (1,)
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")


Erro ao conectar ao banco de dados: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
dados_filtrados.to_sql('dados_rais', con=engine, index=False, if_exists='append', chunksize=500)